In [80]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
from xgboost import XGBRegressor
import re

In [105]:
#add advanced stats from fantasy pros
def cleanFP(dfwr):
    dfwr['Player'] = dfwr['Player'].str.split("(").str[0].str.strip()
    dfwr = dfwr.drop(columns=['AIR', 'AIR/R'])
    return dfwr
#add basic stats from pro football reference
def cleanPFR(dfwr):
    dfwr['Player'] = dfwr['Player'].str.replace(re.escape('*'), '', regex=True)
    dfwr['Player'] = dfwr['Player'].str.replace(re.escape('+'), '', regex=True).str.strip()
    dfwr['Fpts/G'] = dfwr.apply(lambda row: (row.Rec + row.Yds / 10 + row.TD * 6 + row.Fmb * - 2) / row.G, axis=1)
    dfwr = dfwr.drop(columns=['Tm', 'Pos'])
    return dfwr
#combine advanced and basic stats into one dataframe
def combine(dfwrA, dfwrB):
    dfwr = pd.merge(dfwrA, dfwrB, left_on='Player', right_on='Player', how='inner')
    return dfwr

dfwr22A = pd.read_excel(r"C:\Users\luker\Downloads\Python\WR22.xlsx", 'WR22A')
dfwr22A = cleanFP(dfwr22A)
dfwr22B = pd.read_excel(r"C:\Users\luker\Downloads\Python\WR22.xlsx", 'WR22B')
dfwr22B = cleanPFR(dfwr22B)
dfwr22 = combine(dfwr22A, dfwr22B)
dfwr22

,Rank,Player,G_x,REC,YDS,Y/R_x,YBC,YBC/R,YAC,YAC/R,...,Y/R_y,TD,1D,Succ%,Lng,Y/Tgt,R/G,Y/G,Fmb,Fpts/G
0,1,Justin Jefferson,17,128,1809,14.1,1185,9.3,624,4.9,...,14.1,8,80,57.6,64,9.8,7.5,106.4,0,20.994118
1,2,Davante Adams,17,100,1516,15.2,1023,10.2,493,4.9,...,15.2,14,65,46.1,60,8.4,5.9,89.2,1,19.623529
2,3,Tyreek Hill,17,119,1710,14.4,1226,10.3,484,4.1,...,14.4,7,77,56.5,64,10.1,7.0,100.6,1,19.411765
3,4,A.J. Brown,17,88,1496,17.0,953,10.8,543,6.2,...,17.0,11,59,51.7,78,10.3,5.2,88.0,2,17.623529
4,5,Stefon Diggs,17,110,1455,13.2,1023,9.3,432,3.9,...,13.2,11,74,64.3,53,9.3,6.8,89.3,1,19.681250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,192,Bryan Edwards,4,3,15,5.0,6,2.0,9,3.0,...,5.0,0,0,20.0,10,3.0,0.4,2.1,0,0.642857
173,193,Austin Trammell,2,2,13,6.5,13,6.5,0,0.0,...,6.5,0,1,50.0,8,3.3,0.3,2.2,0,0.550000
174,194,Montrell Washington,15,4,2,0.5,-11,-2.8,13,3.3,...,0.5,0,0,0.0,4,0.2,0.3,0.1,5,-0.386667
175,195,Lance McCutcheon,4,0,0,0.0,0,0.0,0,0.0,...,NaN,0,0,0.0,0,0.0,0.0,0.0,0,0.000000
